In [ ]:
# Import the dependencies.
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [ ]:
#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
#################################################
# Flask Setup
#################################################
app = Flask(__name__)


In [ ]:
#################################################
# Flask Routes
#################################################

#Start at the homepage. List all the available routes.

@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/measurement<br/>"
        f"/api/v1.0/station"
    )


In [ ]:
# Route to retrieve precipitation data for the last 12 months
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Parse the end date into a datetime object
    end_date = parser.parse('2017-08-23')

    # Calculate the start date by subtracting 12 months from the end date
    start_date = end_date - relativedelta.relativedelta(months=12)

    # Query using the calculated start and end dates
    precipitations = session.query(Measurement.prcp, Measurement.date).filter(
        Measurement.date >= start_date, Measurement.date <= end_date).all()


In [ ]:
#Create a list of dictionaries containing the precip and dates
all_precipitations = []

for prcp, date in precipitations:
	precip_dictionary = {}
	precip_dictionary["date"] = date
	precip_dictionary["precipitation"] = pcrp
	all_precipitations.append(precip_dictionary)
	

#Return the JSON representation of your dictionary.
return jsonify(all_precipitations)

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# /api/v1.0/stations

# Return a JSON list of stations from the dataset.


In [ ]:
# /api/v1.0/tobs Query the dates and temperature observations of the most-active station for the previous year of data.

# Return a JSON list of temperature observations for the previous year.

In [ ]:
# #/api/v1.0/<start> and /api/v1.0/<start>/<end>

# Return a JSON list of the minimum temperature, the average temperature, and the maximum temperature for a specified start
#or start-end range.

# For a specified start, calculate TMIN, TAVG, and TMAX for all the dates greater than or equal to the start date.

# For a specified start date and end date, calculate TMIN, TAVG, and TMAX for the dates from the start date
#to the end date, inclusive.